In [1]:
if split(pwd(),Base.Filesystem.path_separator)[end] != "CurricularAnalytics.jl"
    cd("../..")
end
println("Current directory: " * split(pwd(),Base.Filesystem.path_separator)[end])
using Pkg
pkg"activate ."
using CurricularAnalytics

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/915ec9b1c760b4a55fdcd04339125526a31beda4-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/915ec9b1c760b4a55fdcd04339125526a31beda4-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

Current directory: CurricularAnalytics.jl


In [2]:

function create_degree_plan2(curric::Curriculum, name::AbstractString="", create_terms::Function=bin_packing2, additional_courses::Array{Course}=Array{Course,1}();
    min_terms::Int=1, max_terms::Int=1, min_credits_per_term::Int=5, max_credits_per_term::Int=19)
    terms =  create_terms(curric,additional_courses; min_terms=min_terms,max_terms=max_terms, min_credits_per_term=min_credits_per_term,
                                max_credits_per_term=max_credits_per_term)
    return DegreePlan(name, curric, terms)
end

function check_requistes2(curric::Curriculum, index::Int, previous_terms::Array{Int}, current_term::Array{Int})
    req_complete = true
    #find all inneighbors of current node
    inngbr = inneighbors(curric.graph, index)
    for ngbr in inngbr
        req_type = curric.courses[index].requisites[curric.courses[ngbr].id]
        if req_type == pre && !(ngbr in previous_terms)
            req_complete = false 
            break      
        elseif req_type == co && !(ngbr in previous_terms) && !(ngbr in current_term)
            req_complete = false 
            break
        end
    end    
    return req_complete
end
"""
function bin_packing2(curric::Curriculum, additional_courses::Array{Course}=Array{Course,1}(); 
    min_terms::Int=1, max_terms::Int=1, min_credits_per_term::Int=5, max_credits_per_term::Int=19)
    #println("min_terms $min_terms max $max_terms min_credits_per_term $min_credits_per_term")
    #println("max_credits_per_term $max_credits_per_term $total_terms")
    #total number of credits
    curric_total_credit=total_credits(curric)
    #Even though the max credit is set, the algorithim will fill in free courses that optimally fill the term, up to the maximum 
    #credits desired by the student or allowed by the university 
    #round to average credit per term
    #make a function to calculate extra(2 in this case)
    #calculate average credit after each term
    if !("complexity" in keys(curric.metrics))
        complexity(curric)
    end
    sorted_index = sortperm(curric.metrics["complexity"][2], rev=true)
    terms = Array{Term}(undef, min_terms)
    all_applied_courses = Int[]
    for current_term in 1:min_terms
        termclasses = Course[]
        this_term_applied_courses = Int[]
        total_credits_for_current_term = 0
        #Find upper limit of average credit for remaining terms to balance the credit hours of terms
        avrg_credit_remaining = floor(Int, (curric_total_credit + min_terms - current_term)/ (min_terms-current_term + 1))
        #println("avrg_credit_remaining $avrg_credit_remaining")
        #println("$curric_total_credit $avrg_credit_remaining")
        #check if upper limit of average credit hours for remaining terms exceeds the maximum credit 
        #If exceed, there is no possible way of fitting remaining classses.
        #Therefor, try againg after increasing term count
        if avrg_credit_remaining  < max_credits_per_term
            #go through all courses to add in current term according to the complexity score
            for index in sorted_index
                #if current course is already added to the previous terms ignore
                if !(index in all_applied_courses) && !(index in this_term_applied_courses)
                    #find vertex and send vertex id
                    #Control reqs 
                    can_be_added = true
                    if check_requistes2(curric, index, all_applied_courses, this_term_applied_courses)
                        credit_add = curric.courses[index].credit_hours
                        courses_to_add = [index]
                        outnbr = outneighbors(curric.graph, index)
                        for ngbr in outnbr
                            req_type = curric.courses[ngbr].requisites[curric.courses[index].id]
                            if req_type == strict_co
                                can_be_added = check_requistes2(curric, ngbr, all_applied_courses, this_term_applied_courses)
                                if !can_be_added
                                    break
                                end
                                credit_add +=  curric.courses[ngbr].credit_hours
                                push!(courses_to_add,ngbr)
                            end
                        end
                        #add current course if we still have enough credit 
                        if can_be_added && total_credits_for_current_term + credit_add <= avrg_credit_remaining
                            total_credits_for_current_term += credit_add
                            for course_index in courses_to_add
                                push!(termclasses, curric.courses[course_index])
                                #also keep indexes of this term's classes
                                push!(this_term_applied_courses,course_index)
                            end
                        end
                        #control if there is any credit to add other course
                        if total_credits_for_current_term == avrg_credit_remaining
                            break
                        end
                    end
                end       
            end    
            #Substract credits of added courses
            curric_total_credit = curric_total_credit - total_credits_for_current_term
            #Create term
            terms[current_term] = Term(termclasses)
            #Before starting to a new term, add all indexes of this term's classes
            for course_in_term in this_term_applied_courses
                push!(all_applied_courses, course_in_term) 
            end
        end
    end
    if length(all_applied_courses) != length(sorted_index)
        if min_terms < max_terms
            # the following print statement can be uncommented for debugging purposes
            #println("Unable to create a $min_terms term plan, attempting a $(min_terms+1) term plan")
            return bin_packing2(curric, additional_courses; min_terms=min_terms+1, max_terms=max_terms, min_credits_per_term=min_credits_per_term,
             max_credits_per_term=max_credits_per_term)
        else 
            println("Unable to create visualization")
            return false
        end
    end
    return terms
end
"""

UndefVarError: UndefVarError: min_terms not defined

In [31]:
using LightGraphs
function create_terms(curric::Curriculum; term_count::Int, min_credits_per_term::Int=1, max_credits_per_term::Int=19)
    if !("complexity" in keys(curric.metrics))
        complexity(curric)
    end
    sorted_index = sortperm(curric.metrics["complexity"][2], rev=true)
    terms = Array{Term}(undef, term_count)
    curric_total_credit=total_credits(curric)
    added_credits = 0
    all_applied_courses = Int[]
    for current_term in 1:term_count
        termclasses = Course[]
        this_term_applied_courses = Int[]
        total_credits_for_current_term = 0
        #check if remaining creadits can be added to the remaining terms on ful load
        if (curric_total_credit-added_credits) <= ((term_count-current_term+1)*max_credits_per_term)
            for index in sorted_index
                if !(index in all_applied_courses) && !(index in this_term_applied_courses)
                    can_be_added = true
                    if check_requistes2(curric, index, all_applied_courses, this_term_applied_courses)
                        credit_add = curric.courses[index].credit_hours
                        courses_to_add = [index] 
                        for ngbr in outneighbors(curric.graph, index)
                            req_type = curric.courses[ngbr].requisites[curric.courses[index].id]
                            if req_type == strict_co
                                can_be_added = check_requistes2(curric, ngbr, all_applied_courses, this_term_applied_courses)
                                if !can_be_added
                                    break
                                end
                                credit_add +=  curric.courses[ngbr].credit_hours
                                push!(courses_to_add,ngbr)
                            end
                        end
                        if can_be_added && total_credits_for_current_term + credit_add <= max_credits_per_term
                            total_credits_for_current_term += credit_add
                            for course_index in courses_to_add
                                push!(termclasses, curric.courses[course_index])
                                push!(this_term_applied_courses,course_index)
                            end
                        end
                        #Check if current term is full
                        if total_credits_for_current_term == max_credits_per_term
                            #There is no more space for any other course
                            break
                        end
                    end
                end       
            end    
            added_credits += total_credits_for_current_term
            terms[current_term] = Term(termclasses)
            for course in this_term_applied_courses
                push!(all_applied_courses, course) 
            end
        else
            return nothing, false
        end
    end
    if length(all_applied_courses) == length(sorted_index)
        return terms, true
    else
        return nothing, false
    end    
end

function fin_min_terms(curric::Curriculum, additional_courses::Array{Course}=Array{Course,1}(); 
    min_terms::Int=1, max_terms::Int=10, min_credits_per_term::Int=1, max_credits_per_term::Int=19)
    for term_count in range(min_terms,max_terms)
        terms, control = create_terms(curric; term_count=term_count, min_credits_per_term = min_credits_per_term,
            max_credits_per_term = max_credits_per_term)
        if control
            return true, terms, term_count
        end
    end
    println("Unable to create visualization for provided maximum term count")
    return false, nothing, nothing
end

function balance_terms(curric::Curriculum, additional_courses::Array{Course}=Array{Course,1}(); 
    term_count::Int=1, min_credits_per_term::Int=1, max_credits_per_term::Int=19)
    for max_credit in range(min_credits_per_term, length=max_credits_per_term-min_credits_per_term+1)
        terms, control = create_terms(curric; term_count=term_count,  min_credits_per_term = min_credits_per_term,
            max_credits_per_term = max_credit)
        if control
            return true, terms, max_credit
        end
    end
    println("Unable to create visualization for provided maximum term count")
    return false, nothing, nothing
end

function bin_packing2(curric::Curriculum, additional_courses::Array{Course}=Array{Course,1}(); 
        min_terms::Int=1, max_terms::Int=1, min_credits_per_term::Int=5, max_credits_per_term::Int=19)
    control, terms, min_term_count = fin_min_terms(curric, additional_courses; min_terms = min_terms,max_terms = max_terms, min_credits_per_term = min_credits_per_term, max_credits_per_term = max_credits_per_term)
    if control
        control_balance, terms, max_credit = balance_terms(curric, additional_courses;
            term_count=min_term_count, min_credits_per_term = min_credits_per_term, max_credits_per_term=max_credits_per_term)
        if control_balance
            return terms
        end
    end
    return false
end


bin_packing2 (generic function with 2 methods)

In [32]:
curric = read_csv("examples/UKY_EE_curric.csv")
#control, terms, min_term_count = min_terms(curric; max_terms = 9, max_credits_per_term = 19)
#println(min_term_count)

terms = bin_packing2(curric; max_terms = 9, max_credits_per_term =19)
visualize(DegreePlan("", curric, terms); notebook=true)

WebIO.Scope("scope-13066dfb-76eb-4a61-837f-b07078719a5a", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src=>"http://localhost:8156",:id=>"curriculum",:events=>Dict(:load=>JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"University of Kentucky EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MA 113\",\"metrics\":{\"complexity\":23.0,\"centrality\":0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"Calculus I\",\"id\":3389674827,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2865398329,\"type\":\"coreq\"}],\"name\":\"PHY 231\",\"metrics\":{\"complexity\":16.0,\"centrality\":62,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"General University Physics\",\"id\":2865398329,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 102\",\"metrics\":{\"complexity\":6.0,\"centrality\":0,\"blocking factor\":3,\"delay factor\":3.0},\"nameSub\":\"Fundamentals of Engineering Computing\",\"id\":502542156,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHE 105\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"General College Chemistry I\",\"id\":738464693,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":746915474,\"type\":\"coreq\"}],\"name\":\"PHY 241\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"General University Physics Lab\",\"id\":746915474,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS/WRD 110\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Composition and Communication I\",\"id\":858428248,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 101\",\"metrics\":{\"complexity\":1.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Engineering Exploration I\",\"id\":2071595408,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2966989896,\"type\":\"prereq\"}],\"name\":\"MA 114\",\"metrics\":{\"complexity\":19.0,\"centrality\":102,\"blocking factor\":11,\"delay factor\":8.0},\"nameSub\":\"Calculus II\",\"id\":2966989896,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":502542156,\"target_id\":1416959221,\"type\":\"prereq\"}],\"name\":\"EE 282\",\"metrics\":{\"complexity\":4.0,\"centrality\":3,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Digital Logic Design\",\"id\":1416959221,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EE 490\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"EE Capstone Design\",\"id\":1116551982,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":502542156,\"target_id\":1123828828,\"type\":\"prereq\"},{\"source_id\":3389674827,\"target_id\":1123828828,\"type\":\"coreq\"},{\"source_id\":2865398329,\"target_id\":1123828828,\"type\":\"coreq\"},{\"source_id\":738464693,\"target_id\":1123828828,\"type\":\"coreq\"}],\"name\":\"EGR 103\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"Engineering Exploration II\",\"id\":1123828828,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CS 215\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Intro. to Program Design Abstraction and Problem Solving\",\"id\":1332377596,\"credits\":4,\"nameCanonical\":\"\"},{\"curri

In [27]:
curric = read_csv("examples/UKY_EE_curric.csv")

Curriculum(3309643389, "University of Kentucky EE Program", "", BS::Degree = 4, semester::System = 0, "26.0101", Course[Course(16242192, Dict(3309643389=>1), "Intro. to Electronics", 3, "EE", "461G", "", "", Dict(2281989961=>pre), LearningOutcome[], Dict{String,Any}()), Course(24551683, Dict(3309643389=>2), "General University Physics", 4, "PHY", "232", "", "", Dict(2865398329=>pre,2193700744=>co), LearningOutcome[], Dict{String,Any}()), Course(109586583, Dict(3309643389=>3), "Supportive Elective", 3, "", "", "", "", Dict{Int64,Requisite}(), LearningOutcome[], Dict{String,Any}()), Course(147960851, Dict(3309643389=>4), "Math/Statistics Elective", 3, "", "", "", "", Dict{Int64,Requisite}(), LearningOutcome[], Dict{String,Any}()), Course(448709314, Dict(3309643389=>5), "Elective EE Laboratory 1", 2, "", "", "", "", Dict{Int64,Requisite}(), LearningOutcome[], Dict{String,Any}()), Course(502542156, Dict(3309643389=>6), "Fundamentals of Engineering Computing", 2, "EGR", "102", "", "", Dict{

In [28]:
out = min_terms(curric; max_terms=20,max_credits_per_term=19)
dp = DegreePlan("sadas",curric,out[2])
visualize(dp; notebook=true)

WebIO.Scope("scope-52d0f5d0-59e6-4ede-afe5-1452d9ef8815", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src=>"http://localhost:8156",:id=>"curriculum",:events=>Dict(:load=>JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"University of Kentucky EE Program\",\"dp_name\":\"sadas\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MA 113\",\"metrics\":{\"complexity\":23.0,\"centrality\":0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"Calculus I\",\"id\":3389674827,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2865398329,\"type\":\"coreq\"}],\"name\":\"PHY 231\",\"metrics\":{\"complexity\":16.0,\"centrality\":62,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"General University Physics\",\"id\":2865398329,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 102\",\"metrics\":{\"complexity\":6.0,\"centrality\":0,\"blocking factor\":3,\"delay factor\":3.0},\"nameSub\":\"Fundamentals of Engineering Computing\",\"id\":502542156,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHE 105\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"General College Chemistry I\",\"id\":738464693,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":746915474,\"type\":\"coreq\"}],\"name\":\"PHY 241\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"General University Physics Lab\",\"id\":746915474,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS/WRD 110\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Composition and Communication I\",\"id\":858428248,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 101\",\"metrics\":{\"complexity\":1.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Engineering Exploration I\",\"id\":2071595408,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2966989896,\"type\":\"prereq\"}],\"name\":\"MA 114\",\"metrics\":{\"complexity\":19.0,\"centrality\":102,\"blocking factor\":11,\"delay factor\":8.0},\"nameSub\":\"Calculus II\",\"id\":2966989896,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":502542156,\"target_id\":1416959221,\"type\":\"prereq\"}],\"name\":\"EE 282\",\"metrics\":{\"complexity\":4.0,\"centrality\":3,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Digital Logic Design\",\"id\":1416959221,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EE 490\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"EE Capstone Design\",\"id\":1116551982,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":502542156,\"target_id\":1123828828,\"type\":\"prereq\"},{\"source_id\":3389674827,\"target_id\":1123828828,\"type\":\"coreq\"},{\"source_id\":2865398329,\"target_id\":1123828828,\"type\":\"coreq\"},{\"source_id\":738464693,\"target_id\":1123828828,\"type\":\"coreq\"}],\"name\":\"EGR 103\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"Engineering Exploration II\",\"id\":1123828828,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CS 215\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Intro. to Program Design Abstraction and Problem Solving\",\"id\":1332377596,\"credits\":4,\"nameCanonical\":\"\"},{\"

In [29]:
out = balance_terms(curric; term_count=9,max_credits_per_term=19)
dp = DegreePlan("sadas",curric,out[2])
visualize(dp; notebook=true)

WebIO.Scope("scope-30df01d5-c674-47a0-80f4-35592f99a584", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src=>"http://localhost:8156",:id=>"curriculum",:events=>Dict(:load=>JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"University of Kentucky EE Program\",\"dp_name\":\"sadas\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MA 113\",\"metrics\":{\"complexity\":23.0,\"centrality\":0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"Calculus I\",\"id\":3389674827,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2865398329,\"type\":\"coreq\"}],\"name\":\"PHY 231\",\"metrics\":{\"complexity\":16.0,\"centrality\":62,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"General University Physics\",\"id\":2865398329,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 102\",\"metrics\":{\"complexity\":6.0,\"centrality\":0,\"blocking factor\":3,\"delay factor\":3.0},\"nameSub\":\"Fundamentals of Engineering Computing\",\"id\":502542156,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHE 105\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"General College Chemistry I\",\"id\":738464693,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":746915474,\"type\":\"coreq\"}],\"name\":\"PHY 241\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"General University Physics Lab\",\"id\":746915474,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2966989896,\"type\":\"prereq\"}],\"name\":\"MA 114\",\"metrics\":{\"complexity\":19.0,\"centrality\":102,\"blocking factor\":11,\"delay factor\":8.0},\"nameSub\":\"Calculus II\",\"id\":2966989896,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":502542156,\"target_id\":1416959221,\"type\":\"prereq\"}],\"name\":\"EE 282\",\"metrics\":{\"complexity\":4.0,\"centrality\":3,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Digital Logic Design\",\"id\":1416959221,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS/WRD 110\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Composition and Communication I\",\"id\":858428248,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EE 490\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"EE Capstone Design\",\"id\":1116551982,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 101\",\"metrics\":{\"complexity\":1.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Engineering Exploration I\",\"id\":2071595408,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2966989896,\"target_id\":2193700744,\"type\":\"prereq\"}],\"name\":\"MA 213\",\"metrics\":{\"complexity\":18.0,\"centrality\":102,\"blocking factor\":10,\"delay factor\":8.0},\"nameSub\":\"Calculus III\",\"id\":2193700744,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":24551683,\"type\":\"prereq\"},{\"source_id\":2193700744,\"target_id\":24551683,\"type\":\"coreq\"}],\"name\":\"PHY 232\",\"metrics\":{\"complexity\":15.0,\"centrality\":121,\"blocking factor\":7,\"delay factor\":8.0},\"nameSub\":\"General University Physics\",\"id\":24551683,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":24551683,\"target_

In [30]:
curric = read_csv("examples/UKY_EE_curric.csv")
control, terms, min_term_count = min_terms(curric; max_terms=20,max_credits_per_term=15)
println(control)
if control
    control_balance, terms, max_credit = balance_terms(curric; term_count=min_term_count,max_credits_per_term=18)
    println(min_term_count)
    if control_balance
        dp = DegreePlan("sadas",curric,terms)
        println(max_credit)
        visualize(dp; notebook=true)
    end
end

true
9
15


WebIO.Scope("scope-68f85416-fafa-4932-a768-fd8e68179a6e", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src=>"http://localhost:8156",:id=>"curriculum",:events=>Dict(:load=>JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"University of Kentucky EE Program\",\"dp_name\":\"sadas\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MA 113\",\"metrics\":{\"complexity\":23.0,\"centrality\":0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"Calculus I\",\"id\":3389674827,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2865398329,\"type\":\"coreq\"}],\"name\":\"PHY 231\",\"metrics\":{\"complexity\":16.0,\"centrality\":62,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"General University Physics\",\"id\":2865398329,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 102\",\"metrics\":{\"complexity\":6.0,\"centrality\":0,\"blocking factor\":3,\"delay factor\":3.0},\"nameSub\":\"Fundamentals of Engineering Computing\",\"id\":502542156,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHE 105\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"General College Chemistry I\",\"id\":738464693,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":746915474,\"type\":\"coreq\"}],\"name\":\"PHY 241\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"General University Physics Lab\",\"id\":746915474,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2966989896,\"type\":\"prereq\"}],\"name\":\"MA 114\",\"metrics\":{\"complexity\":19.0,\"centrality\":102,\"blocking factor\":11,\"delay factor\":8.0},\"nameSub\":\"Calculus II\",\"id\":2966989896,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":502542156,\"target_id\":1416959221,\"type\":\"prereq\"}],\"name\":\"EE 282\",\"metrics\":{\"complexity\":4.0,\"centrality\":3,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Digital Logic Design\",\"id\":1416959221,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS/WRD 110\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Composition and Communication I\",\"id\":858428248,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EE 490\",\"metrics\":{\"complexity\":3.0,\"centrality\":0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"EE Capstone Design\",\"id\":1116551982,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 101\",\"metrics\":{\"complexity\":1.0,\"centrality\":0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Engineering Exploration I\",\"id\":2071595408,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2966989896,\"target_id\":2193700744,\"type\":\"prereq\"}],\"name\":\"MA 213\",\"metrics\":{\"complexity\":18.0,\"centrality\":102,\"blocking factor\":10,\"delay factor\":8.0},\"nameSub\":\"Calculus III\",\"id\":2193700744,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":24551683,\"type\":\"prereq\"},{\"source_id\":2193700744,\"target_id\":24551683,\"type\":\"coreq\"}],\"name\":\"PHY 232\",\"metrics\":{\"complexity\":15.0,\"centrality\":121,\"blocking factor\":7,\"delay factor\":8.0},\"nameSub\":\"General University Physics\",\"id\":24551683,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":24551683,\"target_

In [13]:
pwd()

"/Users/orhanabar/Desktop/CA_Project/Opt/CurricularAnalytics.jl"